In [3]:
from pipelearn.models.yolo6D.data.dataset import Pose6DDataset, category_id_mapping
from pipelearn.models.yolo6D.data.vis import Visualize, VisualizationConfig, ColorScheme
import matplotlib.pyplot as plt
from pipelearn.embdata.embdata.sense.world_object import WorldObject
from embdata.coordinate import BBox2D
from embdata.sense.camera import Camera, Intrinsics, Extrinsics
from embdata.geometry import Transform3D
import numpy as np
from embdata.sense.world import World
from embdata.sense.image import Image as MBImage
from embdata.sense.object_segmentation_agent import SegmentationAgent

def create_world_from_sample(sample):
    segmentation_agent = SegmentationAgent()
    
    # Add image to world
    image = sample['image'].permute(1, 2, 0).numpy()
    
    # Create a World object
    world = World(image=MBImage(array=image))

    camera_info = sample['camera']

    # Add camera to world
    camera = Camera(
        intrinsic=Intrinsics(
            fx=camera_info['intrinsic']['fx'],
            fy=camera_info['intrinsic']['fy'],
            cx=camera_info['intrinsic']['cx'],
            cy=camera_info['intrinsic']['cy'],
        ),
        extrinsic=Extrinsics(
            rotation=camera_info['extrinsic']['rotation'],
            translation=camera_info['extrinsic']['translation'],
        )
    )
    
    world.camera = camera

    world_objects = []
    
    # Add objects to world
    for box, label, rotation, tz, cx_cy in zip(
        sample['bboxes'],
        sample['labels'],
        sample['rotations'],
        sample['tzs'],
        sample['cx_cys'],
    ):
        
        # Set object name/category
        category_id = label.item()
        
        # Set 2D bounding box
        cx, cy, w, h = box
        x1 = (cx - w/2) * image.shape[1]
        y1 = (cy - h/2) * image.shape[0]
        x2 = (cx + w/2) * image.shape[1]
        y2 = (cy + h/2) * image.shape[0]
        bbox_2d = BBox2D(x1=x1, y1=y1, x2=x2, y2=y2)

        cx = cx_cy[0] * image.shape[1]
        cy = cx_cy[1] * image.shape[0]
        tz = tz

        cx = cx_cy[0] * image.shape[1]  # Image coordinates
        cy = cx_cy[1] * image.shape[0]
        # Create homogeneous coordinates [cx, cy, 1]
        point_2d = np.array([cx, cy, 1])

        # Get camera intrinsics
        K = np.array([
            [camera_info['intrinsic']['fx'], 0, camera_info['intrinsic']['cx']],
            [0, camera_info['intrinsic']['fy'], camera_info['intrinsic']['cy']],
            [0, 0, 1]
        ])

        # Calculate 3D point: t = K^-1 * tz * [cx, cy, 1]^T
        point_3d = tz * np.linalg.inv(K) @ point_2d

        tx = point_3d[0].item()
        ty = point_3d[1].item()
        tz = point_3d[2].item()

        try:
            _, mask = segmentation_agent.act(world.image, bbox_2d)
        except Exception as e:
            print("Failed for object: ", bbox_2d)
            continue

        wo = WorldObject(
            name=category_id_mapping[str(label.item())]['category_name'],
            bbox_2d=bbox_2d,
            pose=Transform3D(rotation=rotation.numpy(), translation=np.array([tx, ty, tz])).pose(),
        )
        wo.mask = mask
        world.add_object(wo)
        
    
    return world

def visualize_dataset_samples():
    # Initialize dataset and visualization
    dataset = Pose6DDataset(split="train")
    vis_config = VisualizationConfig(
        masks=False,
        labels=True,
        bbox2d=False,
        bbox3d=True,
        axes=True,
        font_scale=0.7,  # Slightly larger font
        line_thickness=2,
    )
    visualizer = Visualize(config=vis_config)
    
    # Process first few samples
    for idx in range(3):  # Show first 3 images
        sample = dataset[idx]
        
        # Create world object from sample
        world = create_world_from_sample(sample)
        
        # Visualize using the Visualize class
        annotated_image = visualizer.show(world)
        
        # Display the image
        plt.figure(figsize=(15, 10))
        plt.imshow(annotated_image)
        plt.axis('off')
        plt.show()

In [6]:
visualize_dataset_samples()

Resolving data files:   0%|          | 0/78 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/78 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/72 [00:00<?, ?it/s]

Loaded as API: https://api.mbodi.ai/sense/ ✔


[12/19/24 23:25:40] INFO     HTTP Request: GET https://api.mbodi.ai/sense/config "HTTP/1.1 200 OK"  ]8;id=339209;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=399790;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py#1038\1038]8;;\

...

                    INFO     HTTP Request: GET                                                      ]8;id=709927;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=478451;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py#1038\1038]8;;\
                             https://api.mbodi.ai/sense/queue/data?session_hash=da28f286-6e8b-46a2-                
                             83b7-b92fe008ee40 "HTTP/1.1 200 OK"                                                   

                    INFO     HTTP Request: GET                                                      ]8;id=311166;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=518808;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py#1038\1038]8;;\
                             https://api.mbodi.ai/sense/file=/tmp/gradio/5e5a04a11e742cad0efc468149                
                             7f59f94b3599628b43828841e8387b9103ef76/image.webp "HTTP/1.1 200 OK"                   

                    INFO     HTTP Request: GET                                                      ]8;id=280369;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=830333;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py#1038\1038]8;;\
                             https://api.mbodi.ai/sense/file=/tmp/gradio/5e5a04a11e742cad0efc468149                
                             7f59f94b3599628b43828841e8387b9103ef76/image.webp "HTTP/1.1 200 OK"                   

                    INFO     HTTP Request: GET                                                      ]8;id=657627;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=598163;file:///home/tilak/projects/learning/.venv/lib/python3.11/site-packages/httpx/_client.py#1038\1038]8;;\
                             https://api.mbodi.ai/sense/file=/tmp/gradio/5e5a04a11e742cad0efc468149                
                             7f59f94b3599628b43828841e8387b9103ef76/image.webp "HTTP/1.1 200 OK"                   

Object: glue
Pose: Pose6D(
│   'x': 17.338592367339515,
│   'y': 56.3638763046614,
│   'z': 1472.7637939453125,
│   'roll': 0.0,
│   'pitch': 0.0,
│   'yaw': 0.0
)
Bbox 3D: None
Bbox 2D: BBox2D('x1': 330.5, 'y1': 249.5, 'x2': 333.5000305175781, 'y2': 278.5)
